In [102]:
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.ml.feature.OneHotEncoderEstimator
import scala.collection.JavaConverters._ 
import scala.util.hashing.{ MurmurHash3 => MH3 }
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions.row_number
import org.apache.spark.sql.types.StringType
import org.apache.spark.graphx._
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.regression.LinearRegressionModel
import org.apache.spark.mllib.regression.LinearRegressionWithSGD
import com.github.fommil.netlib.BLAS

import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.ml.feature.OneHotEncoderEstimator
import scala.collection.JavaConverters._
import scala.util.hashing.{MurmurHash3=>MH3}
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions.row_number
import org.apache.spark.sql.types.StringType
import org.apache.spark.graphx._
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.regression.LinearRegressionModel
import org.apache.spark.mllib.regression.LinearRegressionWithSGD
import com.github.fommil.netlib.BLAS


In [57]:
val df0 = spark.
        read.
        option("inferSchema", "true").
        option("header","true").
        option("mode","DROPMALFORMED").
        option("delimiter", ",").
        csv("weatherAUS-clean-FINAL.csv")

df0: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 31 more fields]


In [58]:
df0.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- MinTemp: double (nullable = true)
 |-- MaxTemp: double (nullable = true)
 |-- Rainfall: double (nullable = true)
 |-- Evaporation: double (nullable = true)
 |-- Sunshine: double (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- WindGustSpeed: double (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed9am: double (nullable = true)
 |-- WindSpeed3pm: double (nullable = true)
 |-- Humidity9am: double (nullable = true)
 |-- Humidity3pm: double (nullable = true)
 |-- Cloud9am: double (nullable = true)
 |-- Cloud3pm: double (nullable = true)
 |-- Temp9am: double (nullable = true)
 |-- Temp3pm: double (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RISK_MM: double (nullable = true)
 |-- RainTomorrow: string (nullable = true)
 |-- Month: integer (nullable = true)
 |-- RainToday_Sc: integer (nullable = true)
 |--

In [59]:
df0.registerTempTable("ausRainData")

In [60]:
val df1 = spark.sql("select * from ausRainData")

df1: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 31 more fields]


In [61]:
df1.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- MinTemp: double (nullable = true)
 |-- MaxTemp: double (nullable = true)
 |-- Rainfall: double (nullable = true)
 |-- Evaporation: double (nullable = true)
 |-- Sunshine: double (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- WindGustSpeed: double (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed9am: double (nullable = true)
 |-- WindSpeed3pm: double (nullable = true)
 |-- Humidity9am: double (nullable = true)
 |-- Humidity3pm: double (nullable = true)
 |-- Cloud9am: double (nullable = true)
 |-- Cloud3pm: double (nullable = true)
 |-- Temp9am: double (nullable = true)
 |-- Temp3pm: double (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RISK_MM: double (nullable = true)
 |-- RainTomorrow: string (nullable = true)
 |-- Month: integer (nullable = true)
 |-- RainToday_Sc: integer (nullable = true)
 |--

In [62]:
val df10 = df1
    .drop("RainToday").withColumnRenamed("RainToday_Sc", "RainToday")
    .drop("RainTomorrow").withColumnRenamed("RainTomorrow_Sc", "RainTomorrow")
    .drop("Date")
    .drop("WindGustDir")
    .drop("WindDir9am")
    .drop("WindDir3pm")
    .drop("date_Sc")
    .drop("date-day_Sc")
    .drop("date-month_Sc")
    .drop("date-year_Sc")
    .drop("Pressure9am").withColumnRenamed("Pressure9am_Sc","Pressure9am")
    .drop("Pressure3pm").withColumnRenamed("Pressure3pm_Sc","Pressure3pm")

df10: org.apache.spark.sql.DataFrame = [Location: string, MinTemp: double ... 19 more fields]


In [63]:
df10.printSchema()

root
 |-- Location: string (nullable = true)
 |-- MinTemp: double (nullable = true)
 |-- MaxTemp: double (nullable = true)
 |-- Rainfall: double (nullable = true)
 |-- Evaporation: double (nullable = true)
 |-- Sunshine: double (nullable = true)
 |-- WindGustSpeed: double (nullable = true)
 |-- WindSpeed9am: double (nullable = true)
 |-- WindSpeed3pm: double (nullable = true)
 |-- Humidity9am: double (nullable = true)
 |-- Humidity3pm: double (nullable = true)
 |-- Cloud9am: double (nullable = true)
 |-- Cloud3pm: double (nullable = true)
 |-- Temp9am: double (nullable = true)
 |-- Temp3pm: double (nullable = true)
 |-- RISK_MM: double (nullable = true)
 |-- Month: integer (nullable = true)
 |-- RainToday: integer (nullable = true)
 |-- RainTomorrow: integer (nullable = true)
 |-- Pressure3pm: double (nullable = true)
 |-- Pressure9am: double (nullable = true)



In [64]:
 //WARNING: ausRainData is REDEFINED here - YURY
df10.registerTempTable("ausRainData")

In [65]:
val location = df10.select($"Location")
  .flatMap(x => Iterable(x(0).toString))

location: org.apache.spark.sql.Dataset[String] = [value: string]


In [66]:
//I am using org.apache.spark.graphx._ library to crate a hash. - Yury
//Not the intended purpose but achieves the desired outcome - Yury

val hashLookup: RDD[(VertexId, String)] = location 
  .rdd
  .distinct()
  .map(x => (MH3.stringHash(x), x))

hashLookup: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, String)] = MapPartitionsRDD[165] at map at <console>:84


In [72]:
val df2 = hashLookup.toDF     //.show(df2.count().toInt)

df2: org.apache.spark.sql.DataFrame = [_1: bigint, _2: string]


In [73]:
df2.registerTempTable("hashLookup")

In [74]:
val df3 = spark.sql("select * from hashLookup").withColumnRenamed("_1","Hash").withColumnRenamed("_2","LocName")

df3: org.apache.spark.sql.DataFrame = [Hash: bigint, LocName: string]


In [75]:
//WARNING: hashLookup is REDEFINED here - YURY
df3.registerTempTable("hashLookup")

In [76]:
df3.show(df3.count().toInt)

+-----------+----------------+
|       Hash|         LocName|
+-----------+----------------+
| -403901479|    CoffsHarbour|
| -424784333|         Woomera|
| 1735662482|         Bendigo|
| 1885063312|         Mildura|
|-1931965857|        Watsonia|
|-1185911642|           Moree|
|-1501181004|         Penrith|
| 1612109533|      Wollongong|
|  737484909|        Richmond|
| 2126364083|    PerthAirport|
|  220478170|      SalmonGums|
|  461690079|   NorfolkIsland|
| 1374240089|       NorahHead|
|  160510013|        Portland|
| 1038357001|     Witchcliffe|
|-1394747672|          Cairns|
|-1407311822|      WaggaWagga|
|-2085394632|MelbourneAirport|
|-1067979720|       Newcastle|
| -942212088|      PearceRAAF|
|-1075208953|          Sydney|
|-2035869294|       GoldCoast|
|  903326363|        Brisbane|
|-2012030282|          Hobart|
| -213329084|      Launceston|
|  840634082|         Walpole|
|  288157406|    AliceSprings|
| -583384466|        Canberra|
| -365093340|        Dartmoor|
|  33701

In [77]:
val df4 = spark.sql("select RainTomorrow, Hash, MinTemp, MaxTemp, Rainfall, Evaporation, Sunshine, WindGustSpeed, WindSpeed9am, WindSpeed3pm, Humidity9am, Humidity3pm, Cloud9am, Cloud3pm, Temp9am, Temp3pm, RISK_MM, Month, RainToday, Pressure3pm, Pressure9am from ausRainData join hashLookup on Location = LocName")

df4: org.apache.spark.sql.DataFrame = [RainTomorrow: int, Hash: bigint ... 19 more fields]


In [78]:
df4.printSchema()
df4.show()

root
 |-- RainTomorrow: integer (nullable = true)
 |-- Hash: long (nullable = false)
 |-- MinTemp: double (nullable = true)
 |-- MaxTemp: double (nullable = true)
 |-- Rainfall: double (nullable = true)
 |-- Evaporation: double (nullable = true)
 |-- Sunshine: double (nullable = true)
 |-- WindGustSpeed: double (nullable = true)
 |-- WindSpeed9am: double (nullable = true)
 |-- WindSpeed3pm: double (nullable = true)
 |-- Humidity9am: double (nullable = true)
 |-- Humidity3pm: double (nullable = true)
 |-- Cloud9am: double (nullable = true)
 |-- Cloud3pm: double (nullable = true)
 |-- Temp9am: double (nullable = true)
 |-- Temp3pm: double (nullable = true)
 |-- RISK_MM: double (nullable = true)
 |-- Month: integer (nullable = true)
 |-- RainToday: integer (nullable = true)
 |-- Pressure3pm: double (nullable = true)
 |-- Pressure9am: double (nullable = true)

+------------+-----------+-------+-------+--------+-----------+--------+-------------+------------+------------+-----------+-------

In [79]:
val assembler = new VectorAssembler()
  .setInputCols(Array("Hash", "MinTemp", "MaxTemp", "Rainfall", "Evaporation", "Sunshine", "WindGustSpeed", "WindSpeed9am", "WindSpeed3pm", "Humidity9am", "Humidity3pm", "Cloud9am", "Cloud3pm", "Temp9am", "Temp3pm", "RISK_MM", "Month", "RainToday",  "Pressure3pm", "Pressure9am"))
  .setOutputCol("features")

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_788b8269d682


In [80]:
val df5 = assembler.transform(df4)

df5: org.apache.spark.sql.DataFrame = [RainTomorrow: int, Hash: bigint ... 20 more fields]


In [81]:
// Saving data here for future use
df4.coalesce(1)
      .write
     // .option("header","true")
      .option("sep",",")
      .mode("overwrite")
      .csv("data") 

In [82]:
!mv /home/data/*.csv /home/weatherAUS-clean-FINAL-V2.csv

In [83]:
!rm -r /home/data/

YURY ======================================= Preparing data for linear Rgression ===============

In [84]:
val data = sc.textFile("/home/weatherAUS-clean-FINAL-V2.csv")

data: org.apache.spark.rdd.RDD[String] = /home/weatherAUS-clean-FINAL-V2.csv MapPartitionsRDD[220] at textFile at <console>:77


In [117]:
val parsedData = data.map { line =>
  val parts = line.split(',')
  LabeledPoint(parts(0).toDouble, Vectors.dense(parts(1).split(' ').map(_.toDouble)))
}.cache()

parsedData: org.apache.spark.rdd.RDD[org.apache.spark.mllib.regression.LabeledPoint] = MapPartitionsRDD[669] at map at <console>:115


In [108]:
val df7 = parsedData.toDF() // need to view can be removed - YURY

df7: org.apache.spark.sql.DataFrame = [label: double, features: vector]


In [113]:
df7.registerTempTable("myData")

In [114]:
val df8 = spark.sql("select features from myData" )

df8: org.apache.spark.sql.DataFrame = [features: vector]


In [115]:
df8.show()

+----------------+
|        features|
+----------------+
|[-1.394747672E9]|
|[-1.394747672E9]|
|[-1.394747672E9]|
|[-1.394747672E9]|
|[-1.394747672E9]|
|[-1.394747672E9]|
|[-1.394747672E9]|
|[-1.394747672E9]|
|[-1.394747672E9]|
|[-1.394747672E9]|
|[-1.394747672E9]|
|[-1.394747672E9]|
|[-1.394747672E9]|
|[-1.394747672E9]|
|[-1.394747672E9]|
|[-1.394747672E9]|
|[-1.394747672E9]|
|[-1.394747672E9]|
|[-1.394747672E9]|
|[-1.394747672E9]|
+----------------+
only showing top 20 rows



In [103]:
val numIterations = 100
val stepSize = 0.00000001
val model = LinearRegressionWithSGD.train(parsedData, numIterations, stepSize)

numIterations: Int = 100
stepSize: Double = 1.0E-8
model: org.apache.spark.mllib.regression.LinearRegressionModel = org.apache.spark.mllib.regression.LinearRegressionModel: intercept = 0.0, numFeatures = 1


In [104]:
val valuesAndPreds = parsedData.map { point =>
  val prediction = model.predict(point.features)
  (point.label, prediction)
}

valuesAndPreds: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[655] at map at <console>:117


In [105]:
valuesAndPreds.collect().foreach(println)

(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)


(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)


(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)


(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)


(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)


...

(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)


(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)


(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)


(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)


(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(1.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(0.0,NaN)
(1.0,NaN)


In [91]:
val MSE = valuesAndPreds.map{ case(v, p) => math.pow((v - p), 2) }.mean()
println(s"training Mean Squared Error $MSE")

training Mean Squared Error NaN


MSE: Double = NaN


In [96]:
model.save(sc, "scalaLinearRegressionWithSGDModel")

In [97]:
val sameModel = LinearRegressionModel.load(sc, "scalaLinearRegressionWithSGDModel")

sameModel: org.apache.spark.mllib.regression.LinearRegressionModel = org.apache.spark.mllib.regression.LinearRegressionModel: intercept = 0.0, numFeatures = 1


In [98]:
!ssh yury@10.0.2.15 "docker cp bd_cont:/home/scalaLinearRegressionWithSGDModel /home/yury/Downloads/FIT5202-Project/big_data_tutorials/"

In [99]:
!ssh yury@10.0.2.15 "docker cp bd_cont:/home/prepareDataAndRunLR.ipynb /home/yury/Downloads/FIT5202-Project/big_data_tutorials/"

In [118]:
!ssh yury@10.0.2.15 "docker cp bd_cont:/home/weatherAUS-clean-FINAL-V2.csv  /home/yury/Downloads/FIT5202-Project/big_data_tutorials/weatherAUS-clean-FINAL-V2.csv "